# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [1]:

import warnings
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [2]:
import sqlite3
sqliteConnection = sqlite3.connect('.\\sqlite\\go_sales_train.sqlite')
cursor = sqliteConnection.cursor()

<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [3]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(sql_query)
tables = cursor.fetchall()
for table in tables:
    print (table[0])
#Vul dit codeblok verder in

country
order_details
order_header
order_method
product
product_line
product_type
retailer_site
return_reason
returned_item
sales_branch
sales_staff


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [4]:
import pandas as pd
sql_query = "SELECT * FROM sales_staff"
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

sales_staff = pd.DataFrame(rows, columns=column_names)
print(sales_staff.head())

   SALES_STAFF_CODE FIRST_NAME  LAST_NAME                   POSITION_EN  \
0                 4      Denis       Pagé                Branch Manager   
1                 5  Élizabeth     Michel  Level 3 Sales Representative   
2                 6      Émile   Clermont  Level 1 Sales Representative   
3                 7    Étienne     Jauvin  Level 2 Sales Representative   
4                12    Elsbeth  Wiesinger  Level 2 Sales Representative   

          WORK_PHONE  EXTENSION                FAX                   EMAIL  \
0  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60       DPage@grtd123.com   
1  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60     EMichel@grtd123.com   
2  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   EClermont@grtd123.com   
3  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60     EJauvin@grtd123.com   
4  +(49) 40 663 1990     1818.0  +(49) 40 663 4571  EWiesinger@grtd123.com   

   DATE_HIRED  SALES_BRANCH_CODE  
0  1996-11-03                  6  
1  1995-06

## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [5]:

sales_staff['DATE_HIRED'] = pd.to_datetime(sales_staff['DATE_HIRED'])
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Als we hier het jaar uit willen halen, schrijven we:

In [6]:
pd.DatetimeIndex(sales_staff['DATE_HIRED']).quarter

Index([4, 2, 2, 3, 1, 2, 1, 1, 3, 3,
       ...
       3, 1, 4, 1, 3, 4, 4, 4, 4, 1],
      dtype='int32', name='DATE_HIRED', length=102)

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [7]:
sales_staff_code = sales_staff['SALES_STAFF_CODE']
work_phone = sales_staff['WORK_PHONE']
extension = sales_staff['EXTENSION']
fax = sales_staff['FAX']
email = sales_staff['EMAIL']



Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [8]:
contact_detail = pd.concat(
    [sales_staff_code, work_phone, extension, fax, email], axis=1
)

print(contact_detail)

     SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                   4  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60   
1                   5  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60   
2                   6  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   
3                   7  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60   
4                  12  +(49) 40 663 1990     1818.0  +(49) 40 663 4571   
..                ...                ...        ...                ...   
97                120    +32 16 20.73.21     1340.0    +32 16 20.73.32   
98                121    +32 16 20.73.21     1642.0    +32 16 20.73.32   
99                122    +32 16 20.73.21     1633.0    +32 16 20.73.32   
100               123  +(43) 13 79 56 32      325.0  +(43) 13 79 56 33   
101               124  +(43) 13 79 56 32      348.0  +(43) 13 79 56 33   

                      EMAIL  
0         DPage@grtd123.com  
1       EMichel@grtd123.com  
2     EClermont@grtd1

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [9]:
contact_details_5 = contact_detail.head(5)
print(contact_details_5)

   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1818.0  +(49) 40 663 4571   

                    EMAIL  
0       DPage@grtd123.com  
1     EMichel@grtd123.com  
2   EClermont@grtd123.com  
3     EJauvin@grtd123.com  
4  EWiesinger@grtd123.com  


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [10]:
fl_list = ['EN'] * 5
fl_series = pd.Series(fl_list)
contact_details_5['FIRST_LANGUAGE'] = fl_series

print(contact_details_5)

   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1818.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE  
0       DPage@grtd123.com             EN  
1     EMichel@grtd123.com             EN  
2   EClermont@grtd123.com             EN  
3     EJauvin@grtd123.com             EN  
4  EWiesinger@grtd123.com             EN  


Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [11]:
sl_dict = {0: 'FR', 1: 'FR', 2: 'FR', 3:'DE', 4:'DE'}
sl_series = pd.Series(sl_dict)
contact_details_5['SECOND_LANGUAGE'] = sl_series

print(contact_details_5)

   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1818.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE SECOND_LANGUAGE  
0       DPage@grtd123.com             EN              FR  
1     EMichel@grtd123.com             EN              FR  
2   EClermont@grtd123.com             EN              FR  
3     EJauvin@grtd123.com             EN              DE  
4  EWiesinger@grtd123.com             EN              DE  


Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [12]:
tl_dict = {'THIRD_LANGUAGE': ['NL', 'NL', 'JPN', 'JPN', 'KR']}
tl_df = pd.DataFrame(tl_dict)
contact_details_5['THIRD_LANGUAGE'] = tl_df['THIRD_LANGUAGE']

print(contact_details_5)

#Key van de dictionary wordt gebruikt als de naam

   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      325.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      336.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      378.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      398.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1818.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE SECOND_LANGUAGE THIRD_LANGUAGE  
0       DPage@grtd123.com             EN              FR             NL  
1     EMichel@grtd123.com             EN              FR             NL  
2   EClermont@grtd123.com             EN              FR            JPN  
3     EJauvin@grtd123.com             EN              DE            JPN  
4  EWiesinger@grtd123.com             EN              DE             KR  


## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [13]:
new_row = pd.DataFrame(
    {
        'SALES_STAFF_CODE': ['727'],
        'FIRST_NAME': ['TEST'],
        'LAST_NAME': ['TESTLAST'],
        'POSITION_EN': ['Manager'],
        'WORK_PHONE': ['+31 6 12345678'],
        'EXTENSION':['111.0'],
        'FAX': ['+31 6 98765432'],
        'EMAIL': ['newrowmail@gmail.com'],
        'DATE_HIRED': ['2025-02-12']
    }
)
new_row['DATE_HIRED'] = pd.to_datetime(new_row['DATE_HIRED'])
sales_staff = pd.concat([sales_staff, new_row], axis=0, ignore_index=True)

print(sales_staff.tail())

#Axis 0 betekent voeg een rij toe, Axis 1 is voor een kolom

    SALES_STAFF_CODE FIRST_NAME  LAST_NAME                   POSITION_EN  \
98               121   François    De Crée  Level 1 Sales Representative   
99               122     Yvette    Lattrez  Level 3 Sales Representative   
100              123      Willi  Seefelder  Level 2 Sales Representative   
101              124     Sabine     Grüner  Level 3 Sales Representative   
102              727       TEST   TESTLAST                       Manager   

            WORK_PHONE EXTENSION                FAX                   EMAIL  \
98     +32 16 20.73.21    1642.0    +32 16 20.73.32     FDecree@grtd123.com   
99     +32 16 20.73.21    1633.0    +32 16 20.73.32    YLattrez@grtd123.com   
100  +(43) 13 79 56 32     325.0  +(43) 13 79 56 33  WSeefelder@grtd123.com   
101  +(43) 13 79 56 32     348.0  +(43) 13 79 56 33     SGruner@grtd123.com   
102     +31 6 12345678     111.0     +31 6 98765432    newrowmail@gmail.com   

    DATE_HIRED  SALES_BRANCH_CODE  
98  1999-12-01               38.

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [14]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

ValueError: cannot convert float NaN to integer

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.